In [44]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pytz
import openpyxl

#add != Leap Year logic
    # if (year % 4 == 0) and (year % 100 != 0) or (year % 400 == 0):
    #     print('YES')
    # else:
    #     print('NO')

# add 'Local time' MW converting

#Pytz mapping CASE sensitive issues, 
    #BST not parsing - bst parsing...

def get_date_range(file_path, start_row, end_row, target_timezone):

    # Load the workbook 
    workbook = openpyxl.load_workbook(file_path)

    # Select the sheet (active by default, modify if you need specific sheet)
    sheet = workbook.active
    
    # Set column indices for the Specified columns
    description_col = get_column_index(sheet, "Description")
    start_time_col = get_column_index(sheet, "Start Time")
    time_zone_col = get_column_index(sheet, "Time Zone")
    duration_col = get_column_index(sheet, "Duration")

    for row_num in range(start_row, end_row + 1):
        # Get data from specified columns in the current row
        start_description = sheet.cell(row=row_num, column=description_col).value
        print(f"Start Description: {start_description}")
        start_time = sheet.cell(row=row_num, column=start_time_col).value
        print(f"Start Time: {start_time}")
        start_timezone = sheet.cell(row=row_num, column=time_zone_col).value
        print(f"Start Timezone: {start_timezone}")
        duration_hours = sheet.cell(row=row_num, column=duration_col).value
        duration_hours = int(duration_hours.replace(" hours", ""))
        print(f"Duration (hours): {duration_hours}")
        
        converter(start_description, start_time, start_timezone, duration_hours, target_timezone)

def get_column_index(sheet, column_name):
    # Get the column index based on the column name
    for col_num in range(1, sheet.max_column + 1):
        if sheet.cell(row=1, column=col_num).value == column_name:
            return col_num
    raise ValueError(f"Column '{column_name}' not found in the sheet")

def get_pytz_timezone(start_timezone):
    # Mapping friendly time zone names to pytz time zone names
        timezone_mapping = {
        'argentina': 'America/Argentina/Buenos_Aires',
        'pst': 'America/Los_Angeles',
        'pdt': 'America/Los_Angeles',
        'pst/pdt': 'America/Los_Angeles',
        'PST/PDT': 'America/Los_Angeles',
        'CET/CEST': 'Europe/Berlin',
        'cet/cest': 'Europe/Berlin',
        'cet': 'Europe/Berlin',
        'chile': 'America/Santiago',
        'bst/gmt': 'Europe/London',
        'bst': 'Europe/London',
        'BST': 'Europe/London',
        'colombia': 'America/Bogota',
        'Central Time(US)': 'US/Central',
        'AEST': 'Australia/Brisbane',
        'NZDT': 'NZ',
        #add
        }

        # Try to find a direct timezone match
        pytz_timezone = timezone_mapping.get(start_timezone.lower())
        print(f"mapped timezone: {pytz_timezone}")
        return pytz_timezone

def output(start_timezone, start_datetime, end_datetime, target_timezone, start_datetime_target, end_datetime_target):
        
        #test function to print vars
        print(f"\nOriginal Converted date to mapped {start_timezone} Start Time Zone:")
        print(f"Start Date : {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")

        print(f"Converted to {target_timezone} Target Time Zone:")
        print(f"Start Date : {start_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print("")

        return output

def converter(start_description, start_time, start_timezone, duration_hours, target_timezone):
        
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_indices = {day: index for index, day in enumerate(day_names)}
    description_parts = start_description.split()

    start_datetime = None
    end_datetime = None
    
    # Look for correct format of the start_time within string where comma split description by parts
    if ',' in ' '.join(start_time):

        time_parts = start_time.split(', ')
        specified_time = time_parts[0]
        specified_day = time_parts[1]
    # look for weekday in start_description
    elif any(day in description_parts for day in day_names):

        specified_day = next(day for day in day_names if day in description_parts)
        specified_time = start_time

    elif 'Last Day'.lower() in ' '.join(description_parts).lower():

        specified_day = None
        specified_time = start_time
      
    else:

        raise ValueError(f"Column '{start_time}' has wrong format")

    # find current date
    start_time_24h = datetime.strptime(specified_time, "%I%p").strftime("%H:%M")
    current_date = datetime.now()

    # Convert start_timezone to pytz time zone
    start_timezone_pytz = get_pytz_timezone(start_timezone)

    # Finding the next Monday after the first Tuesday (Patch Tuesday)
    first_day_of_month = current_date.replace(day=1)
    first_tuesday = first_day_of_month + timedelta(days=(1 - first_day_of_month.weekday()) % 7)
    patch_tuesday = first_tuesday + timedelta(days=6)
    next_monday = patch_tuesday + timedelta(days=(7 - patch_tuesday.weekday()) % 7)
  

    if 'next'.lower() in ' '.join(description_parts).lower():

        print("Executing 'Next Month' logic...")
        # Extracting the month
        current_date = current_date.replace(day=1) + relativedelta(months=1)

        # Extracting numeric value for weekday selection (1st, 2nd, 3rd, 4th)
        weekday_selection = int(start_description.split()[0][:-len("th")])

        # Calculating the selected weekday of the next month
        first_day_of_next_month = current_date.replace(day=1)
        selected_weekday = day_indices[specified_day.capitalize()]
        current_weekday = first_day_of_next_month.weekday()
        diff = (selected_weekday - current_weekday) % 7
        start_date = first_day_of_next_month + timedelta(days=diff + 7 * (weekday_selection - 1))

        # Constructing start date and time
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        output(start_timezone, start_datetime, end_datetime, target_timezone, start_datetime_target, end_datetime_target)

        return start_datetime_target.strftime("%Y-%m-%d %H:%M:%S"), end_datetime_target.strftime("%Y-%m-%d %H:%M:%S")

    elif 'after'.lower() in ' '.join(description_parts).lower():

        print("Executing 'after Patch Tuesday' logic...")
        
        if description_parts[0] == '0':
            start_day_index = day_indices[specified_day.capitalize()] - next_monday.weekday()
        else:
            # Extracting the week offset
            week_offset = int(description_parts[0][:-2])
            # Calculate the starting day index within the desired week after Patch Tuesday
            start_day_index = (day_indices[specified_day.capitalize()] - next_monday.weekday() + 7) % 7
            # Adjust to the specified week within the month
            start_day_index += week_offset * 7
            # Adjust if the start_day_index is negative
            if start_day_index < 0:
                start_day_index += 7

        # Constructing start date and time
        start_date = next_monday + timedelta(days=start_day_index)
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        output(start_timezone, start_datetime, end_datetime, target_timezone, start_datetime_target, end_datetime_target)

        return start_timezone, start_datetime_target.strftime("%Y-%m-%d %H:%M:%S"), end_datetime_target.strftime("%Y-%m-%d %H:%M:%S")

    elif 'of the month'.lower() in ' '.join(description_parts).lower():

        print("Executing 'of the' logic...")
        # Extracting numeric value for weekday selection (1st, 2nd, 3rd, 4th)
        weekday_selection = int(description_parts[0][:-len("th")])

        # Finding the current month's first Tuesday
        first_day_of_month = current_date.replace(day=1)
        first_tuesday = first_day_of_month + timedelta(days=(1 - first_day_of_month.weekday()) % 7)

        # Finding the desired weekday of the month
        selected_weekday = day_indices[specified_day.capitalize()]
        current_weekday = first_tuesday.weekday()
        diff = (selected_weekday - current_weekday) % 7
        start_date = first_tuesday + timedelta(days=diff + 7 * (weekday_selection - 1))

        # Constructing start date and time
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        output(start_timezone, start_datetime, end_datetime, target_timezone, start_datetime_target, end_datetime_target)

        return start_datetime_target.strftime("%Y-%m-%d %H:%M:%S"), end_datetime_target.strftime("%Y-%m-%d %H:%M:%S")

    elif 'last day'.lower() in ' '.join(description_parts).lower():

        print("Executing 'Last Day' logic...")
        # Constructing start date and time
        start_date = current_date.replace(day=1) + relativedelta(months=1, days=-1)
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        output(start_timezone, start_datetime, end_datetime, target_timezone, start_datetime_target, end_datetime_target)

        return start_datetime_target.strftime("%Y-%m-%d %H:%M:%S"), end_datetime_target.strftime("%Y-%m-%d %H:%M:%S")

    else:
        # Additional cases of weekday logic...
        raise ValueError(f"Column '{start_description}' has wrong format")

if __name__ == "__main__":
    
    target_timezone = 'Europe/London'

    # Get user input for file path and row number
    file_path = input("Enter the Excel file path (with or without quotes): ").strip('\"\'')
    start_row = int(input("Enter the start row: "))
    end_row = int(input("Enter the end row: "))
   
    #if file_path and row_num = 
    get_date_range(file_path, start_row, end_row, target_timezone)
    



Start Description: Last Day
Start Time: 4AM
Start Timezone: cet
Duration (hours): 2
mapped timezone: Europe/Berlin
Executing 'Last Day' logic...

Original Converted date to mapped cet Start Time Zone:
Start Date : 2024-02-29 04:00:00
End Date: 2024-02-29 06:00:00
Converted to Europe/London Target Time Zone:
Start Date : 2024-02-29 03:00:00
End Date: 2024-02-29 05:00:00

